# TP1 – Analyse des données (IFT599 / IFT799)

**Auteurs :**  
-  
-  
-  3

**Date de remise :** 03 octobre 2025  


In [27]:
import pandas as pd
import json
import numpy as np

# 3.1 Préparation des données (25%)

- **Fusion des différents fichiers** (`transactions`, `cards`, `users`, `mcc`, `fraud`)  
  - Clés de jointure utilisées  
  - Vérifier lees colonnes dupliquées et suffixes `_x`, `_y`  (causés par pandas)
  - Expliquer les choix 


- **1. Identification des variables peu pertinentes**  
  - Liste des variables exclues  
  - Justifications (faible variance, redondance, trop de valeurs manquantes, etc.)(liste dans le pdf)  
  - Visualisations ou statistiques descriptives à l’appui  (généralement boxplot)

In [ ]:
"""ÉTape 1 : retirer les  colonnes non essentiels avant le merged (sauver de la mémoire)"""

""" Étape 1.1 : Nettoyer le fichier users_data.csv"""
# Retrait de certaines colonnes :
# pour users_data.csv = adress, latitude, longitude , car ce sont des identifiant non nécessaire
#birth_year, birth_month, car nous avons current_age
# Information pertinent sur les informations de la carte de crédit du client: current_age, gender,per_capita_income, yearly_income, total_debt, credit_score, num_credit_cards
# la clef de jointure pour les autres fichiers : id
users_data = pd.read_csv("data/users_data.csv")
colonnes_conserver = [
    "id",              
    "current_age",
    "gender",
    "per_capita_income",
    "yearly_income",
    "total_debt",
    "credit_score",
    "num_credit_cards",
]

users_data_nettoyer= users_data[colonnes_conserver].copy()


""" Étape 1.2 : Nettoyer le fichier transactions_data.csv"""
transactions = pd.read_csv("data/transactions_data.csv")
# exclusions de colonnes :
# Merchant_id: propre au dataset, aucune valeur explicative
#zip : trop spécifique et modalité uniques, faible valeur statisitques
#merchant_city, pas de pertinence, sauf si on s'interesse en geographie
colonnes_conserver = [
    "id",             # identifiant transaction (clé de jointure avec fraud.json)
    "client_id",      # jointure avec users_data_nettoyer
    "card_id",        # jointure avec cards_nettoyer
    "date",           # why not
    "amount",
    "use_chip",
    "merchant_state", 
    "mcc",             # jointure avec mcc_codes.json  
    "errors"
]

transactions_nettoyer= transactions[colonnes_conserver].copy()

""" Étape 1.3 : Nettoyer le fichier cards_data.csv"""

cards = pd.read_csv("data/cards_data.csv")
# retrait des colonnes card_number, cvv, acct_open_date, year_pin_last_changed
# card_number et cvv sont des données sensibles non nécessaire dans les analyses
#acct_open_date n'est pas une donnée pertinente
#year_pin_last_changed, sauf vouloir faire des corrélations entre fraude et le changement des pins, a retirer
colonnes_conserver = [
    "id",                # clé pour joindre aux transactions (card_id)
    "client_id",         # clé vers users.id
    "card_brand",
    "card_type",
    "has_chip",
    "num_cards_issued",
    "credit_limit",
    "acct_open_date",
    "year_pin_last_changed",
    "card_on_dark_web"
]
cards_nettoyer = cards[colonnes_conserver].copy()



"""Étape 2  : fusion des fichiers """
# 2.1 Fusion transactions + users
merged_df = transactions_nettoyer.merge(
    users_data_nettoyer,
    left_on="client_id", 
    right_on="id", 
    how="left",
    suffixes=("", "_user")
)
#print(merged_df)
user_cols = [c for c in merged_df.columns if c.endswith("_user")]
print("colonne dupliqué: (_user):", user_cols, len(user_cols))
merged_df = merged_df.drop(columns=user_cols)

# 2.2 Fusion avec cards
merged_df = merged_df.merge(
    cards_nettoyer,
    left_on="card_id",
    right_on="id",
    how="inner",
    suffixes=("", "_card")
)

cards_cols = [c for c in merged_df.columns if c.endswith("_card")]
print("colonne dupliqué: (_card):", cards_cols, len(cards_cols))
merged_df = merged_df.drop(columns=cards_cols)



# 2.3 fusion des MCC avec le json (cest un dictionnaire)
with open("data/mcc_codes.json", "r", encoding="utf-8") as f:
    mcc_map = json.load(f)

# S'assurer que les mcc sont des strings pour matcher
merged_df["mcc"] = merged_df["mcc"].astype(str)
merged_df["mcc_description"] = merged_df["mcc"].map(mcc_map)



# Étape 2.4  fusion des fraude 
with open("data/train_fraud_labels.json", "r", encoding="utf-8") as f:
    fraud_map = json.load(f)["target"]   # dict: transaction_id(str) -> "Yes"/"No"

# Choisir la colonne id transaction
tx_id_col = "transaction_id"

# Mappe "Yes"/"No" puis convertit en 1/0
merged_df["fraud"] = (  ## certaines colonnes sont NaN, car ce n'est pas recenser sil est frauduleux ou non
    merged_df["fraud"] = merged_df["fraud"].fillna(0).astype(int)  # je le met en binaire, vu que les algo aime le binaire, mais booleen sont chouettes too
)
print(merged_df)
merged_df.to_csv("data/merged_df.csv", index=False)


colonne dupliqué: (_user): ['id_user'] 1
colonne dupliqué: (_card): ['id_card', 'client_id_card'] 2
                id  client_id  card_id                 date   amount  \
0          7475327       1556     2972  2010-01-01 00:01:00  $-77.00   
1          7475328        561     4575  2010-01-01 00:02:00   $14.57   
2          7475329       1129      102  2010-01-01 00:02:00   $80.00   
3          7475331        430     2860  2010-01-01 00:05:00  $200.00   
4          7475332        848     3915  2010-01-01 00:06:00   $46.41   
...            ...        ...      ...                  ...      ...   
13305910  23761868       1718     2379  2019-10-31 23:56:00    $1.11   
13305911  23761869       1766     2066  2019-10-31 23:56:00   $12.80   
13305912  23761870        199     1031  2019-10-31 23:57:00   $40.44   
13305913  23761873       1986     5443  2019-10-31 23:58:00    $4.00   
13305914  23761874        489     5697  2019-10-31 23:59:00   $12.88   

                    use_chip mercha

- **2. Nettoyage des données**  
  - Valeurs manquantes  
  - Doublons  
  - Types de données  
  - Incohérences  

- **3. Création de variables dérivées**  
  - Heure de transaction  
  - Jour de la semaine  
  - Montant normalisé  
  - Intervalle entre deux transactions  
  - Autres pertinentes  


- **4. Visualisation exploratoire**  
  - Diagrammes en secteurs des top 10 commerçants (revenus & nombre de transactions)  
  - Visualisation des périodes de pointe  
  - Interprétation  


- **Encodage des variables catégorielles**  
  - Codes MCC, types de cartes  
  - Méthodes d’encodage testées (Label vs One-hot)  
  - Justification des choix retenus 

# 3.2  Analyse d’association (20%)


- **1. Préparation des transactions pour Apriori / FP-Growth**  
  - Structure attendue   (apriori vs fp-growth)
  - Préparation des transactions pour être compatibles avec chacune

- **2.Extraction de règles d’association**  
  - Exemples concrets  
  - Patterns observés (habitudes de consommation, fidélité, etc.)

- **3. Codes MCC et types de commerces**  
  - Dépenses élevées  
  - Patterns comportementaux  


- **4. Associations types cartes ↔ commerçants**  
  - Profils d’utilisation  

- **5.Combinaisons liées aux fraudes**  
  - Montants élevés, heures tardives, types de commerçants


- **6. interprétation des métriques**  
  - Support, confiance, lift  
  - Visualisations des règles  

- **7. Choix des seuils (support, confiance)**  
  - Justification en fonction du volume et de l’objectif  

# 3.3 Analyse de séquences (15%)

- **Construction des séquences de transactions**  
  - Par client  
  - Variables d’ordre  (horodatage, type de commer¸cant, montant, etc.)?

- **Application d’algorithmes (PrefixSpan, SPADE, EMMA)**  
  - Parcours typiques identifiés  


- **Habitudes et routines**  
  - Parcours d’achat récurrents  (fréquences, routines ou habitudes récurrentes)


- **Patterns précurseurs de fraude**  
  - Petites transactions → retrait → fraude  
  - Interprétation  par motifs temporels ou comportementaux


- **Évolution du comportement client**  
  - Séquences avec contraintes temporelles 


- **Visualisations**  
  - Graphes de séquences fréquentes  
  - Aide à l’interprétation  

# 3.4 Recommandation sur les associations ou séquences (10%)

- **Objectif : proposer des items complémentaires**  
- **Méthodes testées :**  
  - Règles d’association fréquentes  
  - Séquences typiques  
  - Similarité entre transactions  
- **Implémentation d’un script Python autonome (`recommender.py`)**  
- **Exemple de recommandations générées**  

---

# 3.5 Analyse statistique comparative des classes (10%)

- **Comparaison fraude vs légitime**  
  - Scatter plots (2D, 3D)  
  - Réduction de dimension (ACP, t-SNE, UMAP)  
  - Regroupements et anomalies  


- **Approche univariée**  
  - Sélection de variables pertinentes  
  - Tests statistiques (t-test, Mann-Whitney, KS-test)  
  - Résultats comparés  


- **Approche multivariée**  
  - Hotelling, analyse discriminante linéaire  
  - Séparation des classes  
  - Visualisations complémentaires 

- **Discussion**  
  - Avantages, limites et implications  
  - Robustesse et pertinence des méthodes  

---

# 3.6 Pr´esentation des r´esultats (20%)


- Objectif de chaque méthode (bref rappel)  
- Démarche appliquée  
- Résultats obtenus  
- Commentaires et interprétations  
- Visualisations pertinentes  
- Citations des sources utilisées (obligatoires)  